# Hybrid Serverless-Container ML Model Training
## Variance-Aware Decision Boundary Learning

**Objective:** Train ML models to predict optimal platform (Lambda vs ECS) based on context

**Key Challenge:** Previous models achieved 100% accuracy by memorizing workload_type → label

**Solution:** Variance-aware features force model to learn real decision boundaries

**Target Accuracy:** 75-90% (NOT 100%!)

---

### Notebook Structure:
1. Environment Setup & Data Loading
2. Exploratory Data Analysis
3. Model Training (4 models)
4. Model Evaluation & Comparison
5. Feature Importance Analysis
6. SHAP Analysis (Interpretability)
7. Model Selection & Saving
8. Inference Testing
9. Deployment Preparation

---
## 1. Environment Setup & Data Loading

In [ ]:
# Install required packages (Google Colab)
!pip install -q pandas numpy scikit-learn xgboost lightgbm matplotlib seaborn shap joblib

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import json

# ML Libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import shap
import joblib

warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"Notebook run: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

### Upload Training Data to Colab

**Option 1:** Upload manually
```python
from google.colab import files
uploaded = files.upload()
```

**Option 2:** Mount Google Drive
```python
from google.colab import drive
drive.mount('/content/drive')
```

**Option 3:** Use the file from repository (update path below)

In [ ]:
# OPTION 1: Upload file manually (uncomment to use)
# from google.colab import files
# uploaded = files.upload()
# DATA_FILE = list(uploaded.keys())[0]

# OPTION 2: Use Google Drive (uncomment and update path)
# from google.colab import drive
# drive.mount('/content/drive')
# DATA_FILE = '/content/drive/MyDrive/ml_training_data_variance_v1_20251123_134539.csv'

# OPTION 3: Direct path (update with your file)
DATA_FILE = 'ml_training_data_variance_v1_20251123_134539.csv'

print(f"Data file: {DATA_FILE}")

In [ ]:
# Load data
print("Loading training data...")
df = pd.read_csv(DATA_FILE)

print("\n" + "="*80)
print("DATA LOADED SUCCESSFULLY")
print("="*80)
print(f"Total samples: {len(df):,}")
print(f"Features: {len(df.columns)}")
print(f"\nColumns: {list(df.columns)}")

# Display first few rows
print("\nFirst 5 rows:")
df.head()

---
## 2. Exploratory Data Analysis

Understanding the variance in our data

In [ ]:
# Define feature columns (NO actual performance metrics!)
FEATURE_COLUMNS = [
    'workload_type_encoded',
    'payload_size_kb',
    'time_window_encoded',
    'load_pattern_encoded',
    'hour_of_day',
    'is_weekend',
    'lambda_memory_limit_mb',
    'payload_squared',
    'payload_log',
    'payload_workload_interaction',
    'payload_hour_interaction',
    'payload_time_window_interaction',
    'workload_time_window_interaction',
    'payload_load_pattern_interaction',
    'time_window_load_pattern_interaction'
]

TARGET_COLUMN = 'balanced_optimal'

print("\n" + "="*80)
print("FEATURE VALIDATION")
print("="*80)
print(f"\nPredictive features: {len(FEATURE_COLUMNS)}")
for i, feat in enumerate(FEATURE_COLUMNS, 1):
    print(f"  {i:2d}. {feat}")

print(f"\nTarget variable: {TARGET_COLUMN}")

# Check for data leakage
forbidden_features = ['lambda_latency_ms', 'ecs_latency_ms', 'lambda_cost_usd', 'ecs_cost_usd']
leaked = [f for f in forbidden_features if f in df.columns]
if leaked:
    print(f"\n❌ DATA LEAKAGE DETECTED: {leaked}")
else:
    print("\n✅ NO DATA LEAKAGE - Clean feature set!")

In [ ]:
# Label distribution analysis
print("\n" + "="*80)
print("LABEL DISTRIBUTION ANALYSIS")
print("="*80)

# Overall distribution
label_counts = df[TARGET_COLUMN].value_counts()
print(f"\nOverall Distribution:")
print(f"  Lambda optimal (1): {label_counts.get(1, 0):,} ({label_counts.get(1, 0)/len(df)*100:.1f}%)")
print(f"  ECS optimal (0):    {label_counts.get(0, 0):,} ({label_counts.get(0, 0)/len(df)*100:.1f}%)")

# Distribution by workload type
print(f"\nDistribution by Workload Type:")
workload_stats = df.groupby('workload_type')[TARGET_COLUMN].agg([
    ('count', 'count'),
    ('lambda_optimal', 'sum'),
    ('lambda_pct', lambda x: x.mean() * 100)
]).round(1)
print(workload_stats)

# Variance validation
print(f"\n🎯 Variance Validation (Target: 20-80%):")
for workload, row in workload_stats.iterrows():
    pct = row['lambda_pct']
    if 20 <= pct <= 80:
        status = "✅ GOOD"
    elif pct < 20 or pct > 80:
        status = "⚠️  Borderline"
    else:
        status = "❌ ISSUE"
    print(f"  {workload:25s} {pct:5.1f}% Lambda optimal  {status}")

In [ ]:
# Visualization: Label distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Overall distribution
ax1 = axes[0]
label_counts.plot(kind='bar', ax=ax1, color=['#e74c3c', '#3498db'])
ax1.set_title('Overall Label Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('Label (0=ECS, 1=Lambda)', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.set_xticklabels(['ECS Optimal', 'Lambda Optimal'], rotation=0)
for i, v in enumerate(label_counts):
    ax1.text(i, v + 500, f'{v:,}\n({v/len(df)*100:.1f}%)', ha='center', fontsize=11)

# By workload type
ax2 = axes[1]
workload_pct = df.groupby('workload_type')[TARGET_COLUMN].mean() * 100
workload_pct.plot(kind='barh', ax=ax2, color='#2ecc71')
ax2.set_title('Lambda Optimal % by Workload', fontsize=14, fontweight='bold')
ax2.set_xlabel('Lambda Optimal (%)', fontsize=12)
ax2.set_ylabel('Workload Type', fontsize=12)
ax2.axvline(x=50, color='red', linestyle='--', linewidth=2, alpha=0.7, label='50% threshold')
ax2.axvspan(20, 80, alpha=0.1, color='green', label='Target variance range')
ax2.legend()
for i, v in enumerate(workload_pct):
    ax2.text(v + 1, i, f'{v:.1f}%', va='center', fontsize=11)

plt.tight_layout()
plt.show()

print("\n✅ Variance visualization complete!")

In [ ]:
# Feature distributions
print("\n" + "="*80)
print("FEATURE STATISTICS")
print("="*80)

# Key features
key_features = ['payload_size_kb', 'time_window_encoded', 'load_pattern_encoded', 'workload_type_encoded']
print(f"\nKey Feature Statistics:")
print(df[key_features].describe().round(2))

# Unique values
print(f"\nUnique Values (Variance Check):")
for feat in key_features:
    unique_count = df[feat].nunique()
    print(f"  {feat:30s} {unique_count:3d} unique values")
    if feat == 'payload_size_kb':
        print(f"    Range: {df[feat].min():.1f} - {df[feat].max():.1f} KB")
        print(f"    Unique sizes: {sorted(df[feat].unique())}")

In [ ]:
# Correlation analysis
print("\nGenerating correlation matrix...")

fig, ax = plt.subplots(figsize=(12, 10))
correlation_matrix = df[FEATURE_COLUMNS + [TARGET_COLUMN]].corr()
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0, ax=ax)
ax.set_title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Correlation with target
print(f"\nTop 10 Features Correlated with Target:")
target_corr = correlation_matrix[TARGET_COLUMN].drop(TARGET_COLUMN).sort_values(ascending=False)
print(target_corr.head(10))

print("\n✅ EDA complete!")

---
## 3. Model Training

Training 4 different models:
1. **Random Forest** - Baseline, interpretable
2. **XGBoost** - Expected best performance
3. **LightGBM** - Fast, efficient
4. **Neural Network** - Comparison

**Target:** 75-90% accuracy (NOT 100%!)

In [ ]:
# Prepare data for training
print("\n" + "="*80)
print("PREPARING DATA FOR TRAINING")
print("="*80)

X = df[FEATURE_COLUMNS].copy()
y = df[TARGET_COLUMN].copy()

print(f"\nFeatures (X): {X.shape}")
print(f"Target (y):   {y.shape}")
print(f"\nClass distribution in y:")
print(y.value_counts())

# Check for missing values
missing = X.isnull().sum()
if missing.sum() > 0:
    print(f"\n⚠️  Missing values detected:")
    print(missing[missing > 0])
else:
    print(f"\n✅ No missing values")

In [ ]:
# Train-test split (60-20-20)
print("\nSplitting data: 60% train, 20% validation, 20% test")

# First split: 60% train, 40% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

# Second split: 20% validation, 20% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"\nTrain set:      {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Validation set: {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Test set:       {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Verify stratification
print(f"\nLabel distribution:")
print(f"  Train:      {y_train.mean()*100:.1f}% Lambda optimal")
print(f"  Validation: {y_val.mean()*100:.1f}% Lambda optimal")
print(f"  Test:       {y_test.mean()*100:.1f}% Lambda optimal")

print("\n✅ Data split complete!")

In [ ]:
# Define models
print("\n" + "="*80)
print("DEFINING ML MODELS")
print("="*80)

models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        min_samples_split=10,
        random_state=42,
        n_jobs=-1,
        verbose=0
    ),
    'XGBoost': XGBClassifier(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1,
        verbosity=0
    ),
    'LightGBM': LGBMClassifier(
        n_estimators=200,
        max_depth=10,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    ),
    'Neural Network': MLPClassifier(
        hidden_layer_sizes=(64, 32, 16),
        activation='relu',
        solver='adam',
        max_iter=500,
        random_state=42,
        verbose=False,
        early_stopping=True,
        validation_fraction=0.1
    )
}

print(f"\n{len(models)} models defined:")
for name, model in models.items():
    print(f"  ✓ {name}")

print("\n✅ Models ready for training!")

In [ ]:
# Train all models
print("\n" + "="*80)
print("TRAINING MODELS")
print("="*80)

results = {}

for name, model in models.items():
    print(f"\n{'='*80}")
    print(f"Training: {name}")
    print(f"{'='*80}")
    
    # Train
    print(f"Fitting model...")
    model.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    
    # Probabilities (for ROC-AUC)
    y_train_proba = model.predict_proba(X_train)[:, 1]
    y_val_proba = model.predict_proba(X_val)[:, 1]
    y_test_proba = model.predict_proba(X_test)[:, 1]
    
    # Metrics
    train_acc = accuracy_score(y_train, y_train_pred)
    val_acc = accuracy_score(y_val, y_val_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    
    train_roc_auc = roc_auc_score(y_train, y_train_proba)
    val_roc_auc = roc_auc_score(y_val, y_val_proba)
    test_roc_auc = roc_auc_score(y_test, y_test_proba)
    
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)
    
    # Store results
    results[name] = {
        'model': model,
        'train_acc': train_acc,
        'val_acc': val_acc,
        'test_acc': test_acc,
        'train_roc_auc': train_roc_auc,
        'val_roc_auc': val_roc_auc,
        'test_roc_auc': test_roc_auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'train_test_gap': train_acc - test_acc,
        'y_test_pred': y_test_pred,
        'y_test_proba': y_test_proba
    }
    
    # Print results
    print(f"\n📊 Results:")
    print(f"  Train Accuracy:      {train_acc:.4f} ({train_acc*100:.2f}%)")
    print(f"  Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
    print(f"  Test Accuracy:       {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"  Train-Test Gap:      {train_acc - test_acc:.4f}")
    print(f"\n  Test Precision:      {precision:.4f}")
    print(f"  Test Recall:         {recall:.4f}")
    print(f"  Test F1-Score:       {f1:.4f}")
    print(f"  Test ROC-AUC:        {test_roc_auc:.4f}")
    
    # Validation check
    if test_acc > 0.95:
        print(f"\n  ⚠️  WARNING: Accuracy too high ({test_acc*100:.1f}%) - may still be memorizing!")
    elif test_acc < 0.70:
        print(f"\n  ⚠️  WARNING: Accuracy too low ({test_acc*100:.1f}%) - data quality issue?")
    elif 0.75 <= test_acc <= 0.90:
        print(f"\n  ✅ EXCELLENT: Accuracy in target range ({test_acc*100:.1f}%)")
    else:
        print(f"\n  ✓ Acceptable accuracy ({test_acc*100:.1f}%)")
    
    if train_acc - test_acc < 0.05:
        print(f"  ✅ GOOD: Low overfitting (gap: {(train_acc - test_acc)*100:.2f}%)")
    else:
        print(f"  ⚠️  WARNING: Overfitting detected (gap: {(train_acc - test_acc)*100:.2f}%)")

print("\n" + "="*80)
print("✅ ALL MODELS TRAINED!")
print("="*80)

---
## 4. Model Evaluation & Comparison

In [ ]:
# Comparison table
print("\n" + "="*80)
print("MODEL COMPARISON")
print("="*80)

comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Train Acc': [r['train_acc'] for r in results.values()],
    'Val Acc': [r['val_acc'] for r in results.values()],
    'Test Acc': [r['test_acc'] for r in results.values()],
    'Precision': [r['precision'] for r in results.values()],
    'Recall': [r['recall'] for r in results.values()],
    'F1-Score': [r['f1'] for r in results.values()],
    'ROC-AUC': [r['test_roc_auc'] for r in results.values()],
    'Train-Test Gap': [r['train_test_gap'] for r in results.values()]
})

# Sort by test accuracy
comparison_df = comparison_df.sort_values('Test Acc', ascending=False).reset_index(drop=True)

print("\n")
print(comparison_df.to_string(index=False))

# Best model
best_model_name = comparison_df.iloc[0]['Model']
best_test_acc = comparison_df.iloc[0]['Test Acc']

print(f"\n🏆 Best Model: {best_model_name} (Test Accuracy: {best_test_acc:.4f})")

In [ ]:
# Visualization: Model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Accuracy comparison
ax1 = axes[0, 0]
x = np.arange(len(comparison_df))
width = 0.25
ax1.bar(x - width, comparison_df['Train Acc'], width, label='Train', alpha=0.8)
ax1.bar(x, comparison_df['Val Acc'], width, label='Validation', alpha=0.8)
ax1.bar(x + width, comparison_df['Test Acc'], width, label='Test', alpha=0.8)
ax1.set_xlabel('Model', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_title('Accuracy Comparison (Train/Val/Test)', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(comparison_df['Model'], rotation=15, ha='right')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)
ax1.axhline(y=0.75, color='green', linestyle='--', alpha=0.5, label='Min target (75%)')
ax1.axhline(y=0.90, color='orange', linestyle='--', alpha=0.5, label='Max target (90%)')

# 2. Test metrics comparison
ax2 = axes[0, 1]
metrics_df = comparison_df[['Model', 'Precision', 'Recall', 'F1-Score']].set_index('Model')
metrics_df.plot(kind='bar', ax=ax2, alpha=0.8)
ax2.set_xlabel('Model', fontsize=12)
ax2.set_ylabel('Score', fontsize=12)
ax2.set_title('Test Metrics Comparison', fontsize=14, fontweight='bold')
ax2.set_xticklabels(comparison_df['Model'], rotation=15, ha='right')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# 3. Overfitting analysis
ax3 = axes[1, 0]
colors = ['green' if gap < 0.05 else 'orange' for gap in comparison_df['Train-Test Gap']]
ax3.bar(comparison_df['Model'], comparison_df['Train-Test Gap'], color=colors, alpha=0.8)
ax3.set_xlabel('Model', fontsize=12)
ax3.set_ylabel('Train-Test Gap', fontsize=12)
ax3.set_title('Overfitting Analysis (lower is better)', fontsize=14, fontweight='bold')
ax3.set_xticklabels(comparison_df['Model'], rotation=15, ha='right')
ax3.axhline(y=0.05, color='red', linestyle='--', linewidth=2, alpha=0.7, label='5% threshold')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# 4. ROC-AUC comparison
ax4 = axes[1, 1]
ax4.barh(comparison_df['Model'], comparison_df['ROC-AUC'], alpha=0.8, color='purple')
ax4.set_xlabel('ROC-AUC Score', fontsize=12)
ax4.set_ylabel('Model', fontsize=12)
ax4.set_title('ROC-AUC Comparison', fontsize=14, fontweight='bold')
ax4.axvline(x=0.75, color='green', linestyle='--', alpha=0.5)
ax4.axvline(x=0.90, color='orange', linestyle='--', alpha=0.5)
ax4.grid(axis='x', alpha=0.3)
for i, v in enumerate(comparison_df['ROC-AUC']):
    ax4.text(v + 0.01, i, f'{v:.4f}', va='center')

plt.tight_layout()
plt.show()

print("\n✅ Model comparison visualizations complete!")

In [ ]:
# Confusion matrices
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

for idx, (name, res) in enumerate(results.items()):
    ax = axes[idx]
    cm = confusion_matrix(y_test, res['y_test_pred'])
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, cbar=False)
    ax.set_title(f'{name}\nTest Acc: {res["test_acc"]:.4f}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Predicted', fontsize=11)
    ax.set_ylabel('Actual', fontsize=11)
    ax.set_xticklabels(['ECS (0)', 'Lambda (1)'])
    ax.set_yticklabels(['ECS (0)', 'Lambda (1)'])

plt.tight_layout()
plt.show()

print("\n✅ Confusion matrices generated!")

In [ ]:
# ROC curves
plt.figure(figsize=(10, 8))

for name, res in results.items():
    fpr, tpr, _ = roc_curve(y_test, res['y_test_proba'])
    auc = res['test_roc_auc']
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.4f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=2)
plt.xlabel('False Positive Rate', fontsize=13)
plt.ylabel('True Positive Rate', fontsize=13)
plt.title('ROC Curves - All Models', fontsize=16, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ ROC curves generated!")

---
## 5. Feature Importance Analysis

Understanding which features the model uses to make decisions

In [ ]:
# Feature importance for tree-based models
print("\n" + "="*80)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*80)

tree_models = ['Random Forest', 'XGBoost', 'LightGBM']

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, model_name in enumerate(tree_models):
    if model_name in results:
        model = results[model_name]['model']
        
        # Get feature importances
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            indices = np.argsort(importances)[::-1]
            
            # Print top 10
            print(f"\n{model_name} - Top 10 Features:")
            for i in range(min(10, len(indices))):
                feat_idx = indices[i]
                print(f"  {i+1:2d}. {FEATURE_COLUMNS[feat_idx]:40s} {importances[feat_idx]:.6f} ({importances[feat_idx]*100:.2f}%)")
            
            # Check for dominance
            max_importance = importances[indices[0]]
            if max_importance > 0.40:
                print(f"\n  ⚠️  WARNING: Top feature dominates ({max_importance*100:.1f}%) - may still be lookup table!")
            else:
                print(f"\n  ✅ GOOD: Feature importance distributed (top: {max_importance*100:.1f}%)")
            
            # Plot
            ax = axes[idx]
            top_n = 10
            top_indices = indices[:top_n]
            top_features = [FEATURE_COLUMNS[i] for i in top_indices]
            top_importances = importances[top_indices]
            
            ax.barh(range(top_n), top_importances, alpha=0.8)
            ax.set_yticks(range(top_n))
            ax.set_yticklabels(top_features, fontsize=9)
            ax.set_xlabel('Importance', fontsize=11)
            ax.set_title(f'{model_name}', fontsize=12, fontweight='bold')
            ax.invert_yaxis()
            ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Feature importance analysis complete!")

---
## 6. SHAP Analysis (Interpretability)

Using SHAP (SHapley Additive exPlanations) to understand model predictions

In [ ]:
# SHAP analysis for best model
print("\n" + "="*80)
print("SHAP ANALYSIS")
print("="*80)

best_model = results[best_model_name]['model']

print(f"\nGenerating SHAP values for: {best_model_name}")
print("This may take a few minutes...")

# Sample data for SHAP (use subset for speed)
shap_sample_size = min(1000, len(X_test))
X_shap = X_test.sample(n=shap_sample_size, random_state=42)

# Create explainer
if best_model_name in ['Random Forest', 'XGBoost', 'LightGBM']:
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(X_shap)
    
    # For binary classification, shap_values might be a list
    if isinstance(shap_values, list):
        shap_values = shap_values[1]  # Use positive class
else:
    # For Neural Network, use KernelExplainer (slower)
    explainer = shap.KernelExplainer(best_model.predict_proba, shap.sample(X_train, 100))
    shap_values = explainer.shap_values(X_shap)
    if isinstance(shap_values, list):
        shap_values = shap_values[1]

print("\n✅ SHAP values computed!")

In [ ]:
# SHAP summary plot
print("\nGenerating SHAP summary plot...")

plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_shap, feature_names=FEATURE_COLUMNS, show=False)
plt.title(f'SHAP Summary Plot - {best_model_name}', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n✅ SHAP summary plot complete!")

In [ ]:
# SHAP feature importance (bar plot)
print("\nGenerating SHAP feature importance...")

plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_shap, feature_names=FEATURE_COLUMNS, plot_type="bar", show=False)
plt.title(f'SHAP Feature Importance - {best_model_name}', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n✅ SHAP feature importance complete!")

---
## 7. Model Selection & Saving for AWS Deployment

In [ ]:
# Select best model
print("\n" + "="*80)
print("MODEL SELECTION")
print("="*80)

print(f"\n🏆 Selected Model: {best_model_name}")
print(f"\n📊 Performance Metrics:")
print(f"  Test Accuracy:  {results[best_model_name]['test_acc']:.4f} ({results[best_model_name]['test_acc']*100:.2f}%)")
print(f"  Precision:      {results[best_model_name]['precision']:.4f}")
print(f"  Recall:         {results[best_model_name]['recall']:.4f}")
print(f"  F1-Score:       {results[best_model_name]['f1']:.4f}")
print(f"  ROC-AUC:        {results[best_model_name]['test_roc_auc']:.4f}")
print(f"  Train-Test Gap: {results[best_model_name]['train_test_gap']:.4f}")

# Model size
import sys
model_size_bytes = sys.getsizeof(best_model)
model_size_mb = model_size_bytes / (1024 * 1024)
print(f"\n  Estimated Size: {model_size_mb:.2f} MB")

# Validation
test_acc = results[best_model_name]['test_acc']
if 0.75 <= test_acc <= 0.90:
    print(f"\n✅ SUCCESS: Accuracy in target range (75-90%)")
    print(f"   Model learned meaningful patterns, not memorization!")
elif test_acc > 0.95:
    print(f"\n⚠️  WARNING: Accuracy too high ({test_acc*100:.1f}%)")
    print(f"   Model may still be memorizing workload_type → label")
else:
    print(f"\n✓ Acceptable accuracy ({test_acc*100:.1f}%)")

In [ ]:
# Save model and metadata
print("\n" + "="*80)
print("SAVING MODEL FOR DEPLOYMENT")
print("="*80)

# Create models directory
import os
os.makedirs('models', exist_ok=True)

# Save model
model_filename = 'models/variance_model_best.pkl'
joblib.dump(best_model, model_filename)
print(f"\n✅ Model saved: {model_filename}")

# Save feature columns
features_filename = 'models/feature_columns.json'
with open(features_filename, 'w') as f:
    json.dump(FEATURE_COLUMNS, f, indent=2)
print(f"✅ Features saved: {features_filename}")

# Save metadata
metadata = {
    'model_type': best_model_name,
    'training_date': datetime.now().isoformat(),
    'training_samples': len(X_train),
    'validation_samples': len(X_val),
    'test_samples': len(X_test),
    'num_features': len(FEATURE_COLUMNS),
    'feature_columns': FEATURE_COLUMNS,
    'performance': {
        'train_accuracy': float(results[best_model_name]['train_acc']),
        'val_accuracy': float(results[best_model_name]['val_acc']),
        'test_accuracy': float(results[best_model_name]['test_acc']),
        'precision': float(results[best_model_name]['precision']),
        'recall': float(results[best_model_name]['recall']),
        'f1_score': float(results[best_model_name]['f1']),
        'roc_auc': float(results[best_model_name]['test_roc_auc']),
        'train_test_gap': float(results[best_model_name]['train_test_gap'])
    },
    'model_config': str(best_model.get_params()) if hasattr(best_model, 'get_params') else 'N/A',
    'target_variable': TARGET_COLUMN,
    'class_labels': {'0': 'ECS', '1': 'Lambda'}
}

metadata_filename = 'models/model_metadata.json'
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✅ Metadata saved: {metadata_filename}")

print("\n" + "="*80)
print("MODEL DEPLOYMENT PACKAGE READY")
print("="*80)
print(f"\n📦 Files for AWS Lambda deployment:")
print(f"  1. {model_filename}")
print(f"  2. {features_filename}")
print(f"  3. {metadata_filename}")
print(f"\n💡 Next: Download these files and use with inference script")

In [ ]:
# Download files from Colab (if running on Colab)
print("\n📥 Downloading model files...\n")

try:
    from google.colab import files
    
    files.download(model_filename)
    files.download(features_filename)
    files.download(metadata_filename)
    
    print("\n✅ Files downloaded successfully!")
except:
    print("Not running on Colab - files saved locally")
    print(f"Files location: {os.path.abspath('models/')}")

---
## 8. Inference Testing

Test the saved model to ensure it can make predictions correctly

In [ ]:
# Test model loading and inference
print("\n" + "="*80)
print("INFERENCE TESTING")
print("="*80)

# Load saved model
print("\nLoading saved model...")
loaded_model = joblib.load(model_filename)
print(f"✅ Model loaded: {type(loaded_model).__name__}")

# Load feature columns
with open(features_filename, 'r') as f:
    loaded_features = json.load(f)
print(f"✅ Features loaded: {len(loaded_features)} features")

# Test prediction on sample data
print("\nTesting predictions on 5 random samples...\n")

test_samples = X_test.sample(n=5, random_state=42)
test_labels = y_test.loc[test_samples.index]

for idx, (sample_idx, sample_row) in enumerate(test_samples.iterrows(), 1):
    # Make prediction
    sample_array = sample_row.values.reshape(1, -1)
    prediction = loaded_model.predict(sample_array)[0]
    probability = loaded_model.predict_proba(sample_array)[0]
    actual = test_labels.loc[sample_idx]
    
    # Get sample info
    workload = df.loc[sample_idx, 'workload_type']
    payload = sample_row['payload_size_kb']
    time_window = df.loc[sample_idx, 'time_window']
    load_pattern = df.loc[sample_idx, 'load_pattern']
    
    print(f"Sample {idx}:")
    print(f"  Context: {workload}, {payload:.0f} KB, {time_window}, {load_pattern}")
    print(f"  Prediction: {'Lambda' if prediction == 1 else 'ECS'}")
    print(f"  Confidence: {max(probability)*100:.1f}% (Lambda: {probability[1]*100:.1f}%, ECS: {probability[0]*100:.1f}%)")
    print(f"  Actual:     {'Lambda' if actual == 1 else 'ECS'}")
    print(f"  Correct:    {'✅' if prediction == actual else '❌'}")
    print()

print("✅ Inference testing complete!")

In [ ]:
# Inference time measurement
print("\n" + "="*80)
print("INFERENCE LATENCY MEASUREMENT")
print("="*80)

import time

# Test inference speed
num_predictions = 1000
test_batch = X_test.sample(n=num_predictions, random_state=42)

print(f"\nMeasuring inference time for {num_predictions} predictions...")

start_time = time.time()
predictions = loaded_model.predict(test_batch)
end_time = time.time()

total_time = end_time - start_time
avg_time_ms = (total_time / num_predictions) * 1000

print(f"\n📊 Inference Performance:")
print(f"  Total time:         {total_time:.4f} seconds")
print(f"  Average per request: {avg_time_ms:.4f} ms")
print(f"  Throughput:         {num_predictions/total_time:.0f} predictions/second")

if avg_time_ms < 5:
    print(f"\n✅ EXCELLENT: Inference latency < 5ms (suitable for real-time)")
elif avg_time_ms < 10:
    print(f"\n✅ GOOD: Inference latency < 10ms (acceptable)")
else:
    print(f"\n⚠️  WARNING: Inference latency > 10ms (may need optimization)")

print("\n✅ Latency measurement complete!")

---
## 9. Summary & Next Steps

In [ ]:
# Final summary
print("\n" + "="*80)
print("TRAINING SUMMARY")
print("="*80)

print(f"\n📊 Dataset:")
print(f"  Total samples:      {len(df):,}")
print(f"  Training samples:   {len(X_train):,}")
print(f"  Validation samples: {len(X_val):,}")
print(f"  Test samples:       {len(X_test):,}")
print(f"  Features:           {len(FEATURE_COLUMNS)}")

print(f"\n🏆 Best Model: {best_model_name}")
print(f"  Test Accuracy:      {results[best_model_name]['test_acc']*100:.2f}%")
print(f"  Precision:          {results[best_model_name]['precision']:.4f}")
print(f"  Recall:             {results[best_model_name]['recall']:.4f}")
print(f"  F1-Score:           {results[best_model_name]['f1']:.4f}")
print(f"  ROC-AUC:            {results[best_model_name]['test_roc_auc']:.4f}")
print(f"  Overfitting Gap:    {results[best_model_name]['train_test_gap']*100:.2f}%")

print(f"\n📦 Deployment Files:")
print(f"  Model:              {model_filename}")
print(f"  Features:           {features_filename}")
print(f"  Metadata:           {metadata_filename}")

print(f"\n⚡ Inference Performance:")
print(f"  Average latency:    {avg_time_ms:.4f} ms")
print(f"  Throughput:         {num_predictions/total_time:.0f} pred/sec")

print("\n" + "="*80)
print("✅ TRAINING COMPLETE - MODEL READY FOR AWS DEPLOYMENT")
print("="*80)

print(f"\n🚀 Next Steps:")
print(f"  1. Download model files (variance_model_best.pkl, feature_columns.json, model_metadata.json)")
print(f"  2. Use the AWS Lambda inference script provided")
print(f"  3. Deploy to AWS Lambda for real-time routing")
print(f"  4. Run comparative evaluation (Lambda-only vs ECS-only vs ML-hybrid)")
print(f"  5. Measure actual cost and latency savings")

print(f"\n💡 Key Achievement:")
print(f"  Model accuracy {results[best_model_name]['test_acc']*100:.1f}% indicates real learning!")
if 0.75 <= results[best_model_name]['test_acc'] <= 0.90:
    print(f"  ✅ SUCCESS: Not a lookup table - model learned meaningful decision boundaries!")
else:
    print(f"  ⚠️  Review feature importance to ensure no single feature dominates")

print("\n" + "="*80)